In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf


import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.SparkContext

import org.apache.spark.SparkConf


In [2]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._

In [3]:
var spark = SparkSession
.builder()
.appName("Java Spark SQL basic example")
.config("spark.master", "local")
.getOrCreate();

def sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@467c0788
defined function sc

In [4]:
val df = spark.read.option("header", true).option("delimiter", ",").option("inferSchema",true).csv("homeworkfile2.csv")

df: DataFrame = [device_id: string, country_code: string ... 12 more fields]

In [5]:
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
val sqlContext= new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._
import org.apache.spark.sql.expressions.Window._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@18142839
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSet}

sqlContext: SQLContext = org.apache.spark.sql.SQLContext@6d55e699
import sqlContext.implicits._

import org.apache.spa

In [9]:
// 1. Calculate how many users who used app in X day 
// a) returned the next day
// b) returned 7 day after


// We have partitioned by the device_id and returned the next day (7 day after) of the date,
// if there is no next date for that device, null is returned
// then we have droped all the null values and count distinct device_id

// returned the next day
df.withColumn("date",to_date(col("timestamp")))
.withColumn("day2",lead(col("date"),1).over(Window.partitionBy("device_id").orderBy("date"))).select("device_id","date","day2")
.na.drop().select(countDistinct(col("device_id"))).show()

// returned 7 day after
df.withColumn("date",to_date(col("timestamp")))
.withColumn("day2",lead(col("date"),7).over(Window.partitionBy("device_id").orderBy("date"))).select("device_id","date","day2")
.na.drop().select(countDistinct(col("device_id"))).show()



+-------------------------+
|count(DISTINCT device_id)|
+-------------------------+
|                       98|
+-------------------------+

+-------------------------+
|count(DISTINCT device_id)|
+-------------------------+
|                       94|
+-------------------------+



In [52]:
//6)* what first 5 events users do in their session start (should be used window functions, find about it in the shared link above)
// - 5 events in array and % of users did it


// Select nescessary columns from the main DataFrame

val df1 = df.select("session_id","timestamp","event_type")

// Create columns with lead function to order first 5 events in a session and concat in a list as seperate column
val df2 = df1
.withColumn("event_1",lead(col("event_type"),1).over(Window.partitionBy("session_id").orderBy("timestamp")))
.withColumn("event_2",lead(col("event_type"),2).over(Window.partitionBy("session_id").orderBy("timestamp")))
.withColumn("event_3",lead(col("event_type"),3).over(Window.partitionBy("session_id").orderBy("timestamp")))
.withColumn("event_4",lead(col("event_type"),4).over(Window.partitionBy("session_id").orderBy("timestamp")))
.withColumn("event_5",lead(col("event_type"),5).over(Window.partitionBy("session_id").orderBy("timestamp")))
.withColumn("5events",concat(col("event_1"),lit(","),col("event_2"),lit(","),col("event_3"),lit(","),col("event_4"),lit(","),col("event_5")))
.select("session_id","5events")

// The first 5 events are presented as list in 5events column in first rows of each session
// Here we choose the first row of each session and drop the remaining raws

 
val df3 = df2
.withColumn("rowId", monotonically_increasing_id())
.withColumn("rowId", row_number.over(Window.partitionBy(col("session_id"))
.orderBy(col("rowId").asc)))
.where(col("rowId") === 1).drop("rowId").na.drop()


//Groupby each sequence of 5 events and calculate the rate of appearance 
val df4 = df3.groupBy("5events").agg(count("session_id").as("NofSessions")).sort(desc("NofSessions"))

df4.withColumn("Rate",round(col("NofSessions")/df4.select(sum("NofSessions")).take(1)(0)(0),3)).show(false)



+--------------------------------------------------------------------------------------------------------------+-----------+-----+
|5events                                                                                                       |NofSessions|Rate |
+--------------------------------------------------------------------------------------------------------------+-----------+-----+
|app_load,interstitial_ad_request,banner_ad_request,interstitial_ad_request,banner_ad_request                  |110        |0.095|
|explore_open,interstitial_ad_request,interstitial_ad_request,app_load,native_ad_request                       |107        |0.093|
|explore_open,app_load,interstitial_ad_request,banner_ad_request,interstitial_ad_request                       |54         |0.047|
|app_load,interstitial_ad_request,editor_ad_request,interstitial_ad_request,banner_ad_request                  |32         |0.028|
|explore_open,interstitial_ad_request,app_load,native_ad_request,native_ad_request 

df1: DataFrame = [session_id: string, timestamp: timestamp ... 1 more field]
df2: DataFrame = [session_id: string, 5events: string]
df3: DataFrame = [session_id: string, 5events: string]
df4: Dataset[Row] = [5events: string, NofSessions: bigint]